# TECH Challenge
# Enunciado
Imagine agora, que você vai atuar como Expert em Data Analytics em uma empresa que exporta vinhos do Brasil para o mundo todo.
Sua área é recém-criada dentro da empresa, e você será responsável pelos relatórios iniciais a serem apresentados em uma reunião de investidores e acionistas, explicando a quantidade de vinhos exportados e os fatores externos que podem vir a surgir e que interferem nas análises:
1. Dados climáticos.
2. Dados demográficos.
3. Dados econômicos.
4. Dados de avaliações de vinhos.

O Head de Dados pediu para que você construísse uma tabela contendo as seguintes informações:
1. País de origem (Brasil).
2. País de destino.
3. Quantidade em litros de vinho exportado (utilize: 1KG =1L).
4. Valor em US$.

Os dados que lhe forneceram são de uma vinícola parceira, e podem ser encontrados aqui: http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01
Seu objetivo é dizer o **montante de venda de exportação nos últimos 15 anos**, separando a análise por país e trazendo quais as prospecções futuras e possíveis ações para uma melhoria nas exportações. 


Construa gráficos atraentes e que passem a ideia central para que os acionistas e investidores possam seguir em frente com suas ações.

# Importando bibliotecas

In [11]:
import pandas as pd
import matplotlib as plt
import numpy as np

# Montando os DataFrames iniciais

In [12]:
def convert_NA(df : pd.DataFrame, columns : list = [], na : str = '-') -> pd.DataFrame:
    ''' 
    Takes a list of columns in a dataframe and converts any value 'na' into NaN
    '''
    df_new = df.copy()
    if len(columns) < 1:
        columns = df_new.columns
    df_new[columns] = df_new[columns].replace(na, np.nan)
    return df_new

In [13]:
def convert_float(df : pd.DataFrame, columns : list = [], thousand : str = '.', decimal : str = ',', na : str = '-') -> pd.DataFrame:
    ''' 
    Takes a list of columns in a dataframe and tries to convert them all into float64
    '''
    df_new = df.copy()
    df_new[columns] = df_new[columns].replace('\\.', '', regex=True).replace(decimal, '.', regex=True).astype(float)
    return df_new

In [14]:
def format_export(df: pd.DataFrame, type : str = '', year : int = 0):
    '''
    Takes a DataFrame, adds new columns, reorders them, and converts them to the correct type 
    '''
    df.drop(columns=['Unnamed: 0'], inplace=True)
    df.columns = ['categoria', 'pais_destino', 'kg','valor']
    df['ano'] = year
    df['pais_origem'] = 'Brasil'
    df['tipo'] = type

    columns_ordered = ['tipo','categoria','pais_origem','pais_destino','ano','kg','valor']
    df = df[columns_ordered]
    
    columns_float = ['kg', 'valor']
    df['ano'] = df['ano'].astype(int)

    df = convert_NA(df, columns_float)
    df = convert_float(df, columns_float)

    return df


In [15]:
def concatenate_from_type(file_path : str = './extraidos_final/', file_prefix : str ='exportacao_uvas_frescas', type : str ='Uvas frescas',year_first = 2009, year_last = 2024):
    file_name = file_path + f'{file_prefix}_{year_first}.csv'
    df = pd.read_csv(file_name)
    df = format_export(df, type=type, year=year_first)
    for year in range(year_first+1, year_last+1):
        file_name = file_path + f'{file_prefix}_{year}.csv'
        df_add = pd.read_csv(file_name)
        df_add = format_export(df_add, type=type, year=year)
        df = pd.concat([df, df_add], ignore_index=True)
    return df

In [16]:
df_mesa = concatenate_from_type(file_prefix='exportacao_vinhos_mesa', type='Vinhos de mesa')
df_espumantes = concatenate_from_type(file_prefix='exportacao_vinhos_espumantes', type='Vinhos espumantes')
df_suco = concatenate_from_type(file_prefix='exportacao_suco_de_uva', type='Suco de uva')
df_uvas_frescas = concatenate_from_type(file_prefix = 'exportacao_uvas_frescas', type='Uvas frescas')

In [17]:
df_exports = pd.concat([df_mesa,df_espumantes,df_suco,df_uvas_frescas], ignore_index=True)
df_exports

,tipo,categoria,pais_origem,pais_destino,ano,kg,valor
0,Vinhos de mesa,NaN,Brasil,Afeganistão,2009,NaN,NaN
1,Vinhos de mesa,NaN,Brasil,África do Sul,2009,NaN,NaN
2,Vinhos de mesa,NaN,Brasil,"Alemanha, República Democrática",2009,225086.0,393482.0
3,Vinhos de mesa,NaN,Brasil,Angola,2009,54786.0,84235.0
4,Vinhos de mesa,NaN,Brasil,Anguilla,2009,NaN,NaN
...,...,...,...,...,...,...,...
8203,Uvas frescas,NaN,Brasil,Vanuatu,2024,7.0,16.0
8204,Uvas frescas,NaN,Brasil,Venezuela,2024,20.0,79.0
8205,Uvas frescas,NaN,Brasil,Vietnã,2024,5.0,21.0
8206,Uvas frescas,NaN,Brasil,"Wallis e Futuna, Ilhas",2024,10.0,57.0


In [18]:
df_exports.to_csv('./extraidos/ALL_exports.csv')